Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Device: cuda
📁 Dataset: /content/drive/MyDrive/dev
💾 Output: /content/drive/MyDrive/opus_enhanced_models

🎵 ULTRA-CONSERVATIVE OPUS ENHANCEMENT TRAINING

🎙️ OPUS: 12kbps, complexity=0, frame=160samples
📂 Loading dataset...
🔍 Scanning for audio files...
✅ Found 288 files (limited to 2000).
🧠 Ultra-light model: 1,746 parameters
🛡️ Conservative enhancement (alpha max=0.10)

🚀 STARTING TRAINING



Epoch 1/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 2/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 3/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 4/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 5/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 5...

📈 EPOCH 5 RESULTS:
  PESQ Baseline:    2.194
  PESQ Enhanced:    2.194
  PESQ Improvement: -0.000 ⚠️
  STOI Score:       0.890
  Avg. Latency (for 2.0s chunk): 0.36ms
  Model Alpha:      0.018

💾 Saved best model to /content/drive/MyDrive/opus_enhanced_models/best_model.pth (Improvement: -0.000)


🤔 INTERACTIVE CHECKPOINT - Epoch 5
Current PESQ improvement: -0.000
Continue training? (y/n): y
▶️ Continuing training...



Epoch 6/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 7/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 8/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 9/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 10/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 10...

📈 EPOCH 10 RESULTS:
  PESQ Baseline:    2.304
  PESQ Enhanced:    2.304
  PESQ Improvement: +0.000 ✅
  STOI Score:       0.890
  Avg. Latency (for 2.0s chunk): 0.39ms
  Model Alpha:      0.025

💾 Saved best model to /content/drive/MyDrive/opus_enhanced_models/best_model.pth (Improvement: +0.000)



Epoch 11/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 12/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 13/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 14/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 15/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 15...

📈 EPOCH 15 RESULTS:
  PESQ Baseline:    2.213
  PESQ Enhanced:    2.214
  PESQ Improvement: +0.001 ✅
  STOI Score:       0.888
  Avg. Latency (for 2.0s chunk): 0.40ms
  Model Alpha:      0.032

💾 Saved best model to /content/drive/MyDrive/opus_enhanced_models/best_model.pth (Improvement: +0.001)



Epoch 16/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 17/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 18/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 19/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 20/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 20...

📈 EPOCH 20 RESULTS:
  PESQ Baseline:    2.146
  PESQ Enhanced:    2.151
  PESQ Improvement: +0.005 ✅
  STOI Score:       0.895
  Avg. Latency (for 2.0s chunk): 0.39ms
  Model Alpha:      0.039

💾 Saved best model to /content/drive/MyDrive/opus_enhanced_models/best_model.pth (Improvement: +0.005)



Epoch 21/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 22/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 23/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 24/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 25/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 25...

📈 EPOCH 25 RESULTS:
  PESQ Baseline:    2.281
  PESQ Enhanced:    2.289
  PESQ Improvement: +0.008 ✅
  STOI Score:       0.885
  Avg. Latency (for 2.0s chunk): 0.36ms
  Model Alpha:      0.044

💾 Saved best model to /content/drive/MyDrive/opus_enhanced_models/best_model.pth (Improvement: +0.008)



Epoch 26/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 27/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 28/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 29/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 30/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 30...

📈 EPOCH 30 RESULTS:
  PESQ Baseline:    2.170
  PESQ Enhanced:    2.175
  PESQ Improvement: +0.005 ✅
  STOI Score:       0.887
  Avg. Latency (for 2.0s chunk): 0.36ms
  Model Alpha:      0.049



Epoch 31/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 32/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 33/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 34/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 35/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 35...

📈 EPOCH 35 RESULTS:
  PESQ Baseline:    2.103
  PESQ Enhanced:    2.107
  PESQ Improvement: +0.004 ✅
  STOI Score:       0.896
  Avg. Latency (for 2.0s chunk): 0.36ms
  Model Alpha:      0.052



Epoch 36/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 37/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 38/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 39/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 40/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 40...

📈 EPOCH 40 RESULTS:
  PESQ Baseline:    2.209
  PESQ Enhanced:    2.215
  PESQ Improvement: +0.006 ✅
  STOI Score:       0.890
  Avg. Latency (for 2.0s chunk): 0.37ms
  Model Alpha:      0.053



Epoch 41/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 42/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 43/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 44/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 45/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 45...

📈 EPOCH 45 RESULTS:
  PESQ Baseline:    2.247
  PESQ Enhanced:    2.245
  PESQ Improvement: -0.002 ⚠️
  STOI Score:       0.893
  Avg. Latency (for 2.0s chunk): 0.36ms
  Model Alpha:      0.054



Epoch 46/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 47/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 48/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 49/50:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 50/50:   0%|          | 0/16 [00:00<?, ?it/s]


📊 Evaluating at epoch 50...

📈 EPOCH 50 RESULTS:
  PESQ Baseline:    2.288
  PESQ Enhanced:    2.287
  PESQ Improvement: -0.001 ⚠️
  STOI Score:       0.892
  Avg. Latency (for 2.0s chunk): 0.35ms
  Model Alpha:      0.054

✅ Training complete.


✅ TRAINING COMPLETE!
📁 Best model saved to: /content/drive/MyDrive/opus_enhanced_models/best_model.pth
📈 History plot saved to: /content/drive/MyDrive/opus_enhanced_models/training_history.png



In [11]:
#%%
# ============================================================================
# CELL 1: INSTALLATIONS
# ============================================================================
!pip install -q pyogg sounddevice pesq pystoi librosa torch torchaudio onnx onnxruntime
!apt-get install -qq libopus0 libopus-dev opus-tools
!pip install -q opuslib

#%%
# ============================================================================
# CELL 2: IMPORTS AND SETUP
# ============================================================================
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchaudio
import librosa
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from pesq import pesq
from pystoi import stoi
import opuslib
import soundfile as sf
from google.colab import drive
import warnings
import multiprocessing as mp
import time
import glob

# FIX: Set multiprocessing start method for Colab
try:
    mp.set_start_method('spawn', force=True)
except RuntimeError:
    pass

# Mount Google Drive
drive.mount('/content/drive')
warnings.filterwarnings('ignore')

#%%
# ============================================================================
# CELL 3: CONFIGURATION
# ============================================================================
class Config:
    # Paths
    DATASET_PATH = "/content/drive/MyDrive/dev" # <-- IMPORTANT: Set your dataset path
    OUTPUT_PATH = "/content/drive/MyDrive/opus_enhanced_models" # <-- Models will save here

    # Audio parameters (REAL-TIME OPTIMIZED)
    SAMPLE_RATE = 16000
    CHUNK_DURATION = 2.0 # Train on 2-second segments

    # OPUS parameters - MODERATE DEGRADATION
    OPUS_BITRATE = 12000    # 12kbps
    OPUS_FRAME_SIZE = 160   # 10ms frames = LOW LATENCY (16000 * 0.010)
    OPUS_COMPLEXITY = 0     # MINIMUM complexity for speed

    # Training parameters
    BATCH_SIZE = 16 # Can be larger for small models
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.0001
    WEIGHT_DECAY = 0.0001
    GRAD_CLIP = 0.5
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Quick test mode
    MAX_TRAIN_SAMPLES = 2000 # Max audio clips to load
    EVAL_FREQUENCY = 5
    INTERACTIVE_CHECKPOINT = 5

    # DataLoader parameters
    NUM_WORKERS = 0 # <-- FIX: Set to 0 to avoid multiprocessing/pickling errors

    # Model parameters (ULTRA-LIGHTWEIGHT FOR <15ms LATENCY)
    HIDDEN_SIZE = 32
    KERNEL_SIZE = 3

    # Conservative training
    INITIAL_ALPHA = 0.01  # Start VERY conservative
    MAX_ALPHA = 0.1     # Never go above 10% enhancement

config = Config()
os.makedirs(config.OUTPUT_PATH, exist_ok=True)

print(f"🚀 Device: {config.DEVICE}")
print(f"📁 Dataset: {config.DATASET_PATH}")
print(f"💾 Output: {config.OUTPUT_PATH}")

#%%
# ============================================================================
# CELL 4: OPUS CODEC WRAPPER
# ============================================================================
class OPUSProcessor:
    def __init__(self, sample_rate=16000, bitrate=12000, frame_size=320):
        self.sample_rate = sample_rate
        self.bitrate = bitrate
        self.frame_size = frame_size
        self.encoder = opuslib.Encoder(sample_rate, 1, opuslib.APPLICATION_VOIP)
        self.encoder.bitrate = bitrate
        self.encoder.complexity = config.OPUS_COMPLEXITY
        self.decoder = opuslib.Decoder(sample_rate, 1)
        print(f"🎙️ OPUS: {bitrate/1000:.0f}kbps, complexity={config.OPUS_COMPLEXITY}, frame={frame_size}samples")

    def process(self, audio):
        """Processes an entire audio signal by chunking it."""
        try:
            audio = np.clip(audio, -1.0, 1.0)
            audio_int16 = (audio * 32767).astype(np.int16)
            output_frames = []

            for i in range(0, len(audio_int16), self.frame_size):
                frame = audio_int16[i:i + self.frame_size]

                if len(frame) < self.frame_size:
                    frame = np.pad(frame, (0, self.frame_size - len(frame)))

                encoded = self.encoder.encode(frame.tobytes(), self.frame_size)
                decoded = self.decoder.decode(encoded, self.frame_size)

                if decoded is None:
                    decoded_array = np.zeros(self.frame_size, dtype=np.int16)
                else:
                    decoded_array = np.frombuffer(decoded, dtype=np.int16)

                output_frames.append(decoded_array)

            output = np.concatenate(output_frames)[:len(audio_int16)]
            output_float = output.astype(np.float32) / 32767.0
            output_float = np.clip(output_float, -1.0, 1.0)
            return output_float

        except Exception as e:
            print(f"⚠️ OPUS error: {e}")
            return audio # Return original on error

#%%
# ============================================================================
# CELL 5: ULTRA-CONSERVATIVE ENHANCEMENT NETWORK
# ============================================================================
class UltraLightEnhancer(nn.Module):
    """Ultra-conservative model that barely touches the signal"""
    def __init__(self):
        super().__init__()
        self.enhance = nn.Sequential(
            nn.Conv1d(1, config.HIDDEN_SIZE, kernel_size=config.KERNEL_SIZE, padding=config.KERNEL_SIZE//2),
            nn.PReLU(config.HIDDEN_SIZE, init=0.01),
            nn.Conv1d(config.HIDDEN_SIZE, config.HIDDEN_SIZE//2, kernel_size=config.KERNEL_SIZE, padding=config.KERNEL_SIZE//2),
            nn.PReLU(config.HIDDEN_SIZE//2, init=0.01),
            nn.Conv1d(config.HIDDEN_SIZE//2, 1, kernel_size=1),
        )

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.xavier_uniform_(m.weight, gain=0.01) # Very small initialization
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

        self.alpha = nn.Parameter(torch.tensor(config.INITIAL_ALPHA))

    def forward(self, x):
        identity = x
        enhanced_residual = self.enhance(x)

        # Very conservative mixing (max 10% enhancement)
        alpha = torch.clamp(self.alpha, 0.0, config.MAX_ALPHA)

        # Additive enhancement
        output = (1 - alpha) * identity + alpha * enhanced_residual

        return torch.clamp(output, -1.0, 1.0)

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

#%%
# ============================================================================
# CELL 6: DATASET
# ============================================================================
class AudioEnhancementDataset(Dataset):
    def __init__(self, audio_dir, opus_processor, chunk_duration, sample_rate, max_samples):
        self.audio_files = []
        print("🔍 Scanning for audio files...")
        for ext in ('*.wav', '*.mp3', '*.flac'):
            self.audio_files.extend(glob.glob(os.path.join(audio_dir, "**", ext), recursive=True))

        if not self.audio_files:
            print(f"⚠️ No audio files found in {audio_dir}. Please check config.DATASET_PATH")

        self.audio_files = self.audio_files[:max_samples]
        print(f"✅ Found {len(self.audio_files)} files (limited to {max_samples}).")

        self.opus_processor = opus_processor
        self.sample_rate = sample_rate
        self.chunk_length = int(chunk_duration * sample_rate)

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        try:
            audio, sr = librosa.load(audio_file, sr=self.sample_rate, mono=True)

            if len(audio) < self.chunk_length:
                 # Pad short audio
                audio = np.pad(audio, (0, self.chunk_length - len(audio)))
            else:
                # Take random chunk
                start = np.random.randint(0, len(audio) - self.chunk_length + 1)
                audio = audio[start:start + self.chunk_length]

            # Normalize
            max_val = np.max(np.abs(audio))
            if max_val > 0.01:
                audio = audio / max_val * 0.95

            audio = audio - np.mean(audio) # DC offset removal

            clean = torch.FloatTensor(audio).unsqueeze(0)

            # Run degradation
            degraded_np = self.opus_processor.process(audio)
            degraded = torch.FloatTensor(degraded_np).unsqueeze(0)

            # Ensure no silent/corrupt files
            if torch.all(clean < 1e-3) or torch.all(degraded < 1e-3):
                return self.__getitem__(np.random.randint(0, len(self.audio_files)))

            return degraded, clean

        except Exception as e:
            # print(f"Error loading {audio_file}: {e}")
            return self.__getitem__(np.random.randint(0, len(self.audio_files))) # Retry with another file

#%%
# ============================================================================
# CELL 7: CONSERVATIVE LOSS FUNCTION
# ============================================================================
class ConservativeLoss(nn.Module):
    """Loss that penalizes over-processing"""
    def __init__(self, deviation_weight=0.2):
        super().__init__()
        self.deviation_weight = deviation_weight

    def forward(self, enhanced, clean, degraded):
        # Main loss: get close to clean
        clean_loss = F.l1_loss(enhanced, clean)

        # Penalty: don't deviate too much from degraded (prevent over-processing)
        deviation_penalty = F.l1_loss(enhanced, degraded)

        # *** CRITICAL FIX ***
        # We ADD the penalty. (Original script SUBTRACTED it, rewarding deviation)
        total = clean_loss + (self.deviation_weight * deviation_penalty)

        return total, clean_loss, deviation_penalty

#%%
# ============================================================================
# CELL 8: EVALUATION
# ============================================================================
def evaluate_metrics(model, test_loader, device, max_batches=20):
    model.eval()
    pesq_scores, stoi_scores, pesq_baseline = [], [], []
    latencies = []

    with torch.no_grad():
        for i, (degraded, clean) in enumerate(test_loader):
            if i >= max_batches:
                break

            degraded = degraded.to(device)
            clean_np = clean.numpy()

            # Measure latency
            if device == 'cuda': torch.cuda.synchronize()
            start_time = time.perf_counter()
            enhanced = model(degraded)
            if device == 'cuda': torch.cuda.synchronize()

            # Latency per sample in the batch
            latency = (time.perf_counter() - start_time) * 1000 / degraded.size(0)
            latencies.append(latency)

            # Calculate metrics
            degraded_np = degraded.cpu().numpy()
            enhanced_np = enhanced.cpu().numpy()

            for j in range(len(clean_np)):
                try:
                    clean_sig = clean_np[j, 0]
                    degraded_sig = degraded_np[j, 0]
                    enhanced_sig = enhanced_np[j, 0]

                    pesq_base = pesq(config.SAMPLE_RATE, clean_sig, degraded_sig, 'wb')
                    pesq_enh = pesq(config.SAMPLE_RATE, clean_sig, enhanced_sig, 'wb')
                    pesq_baseline.append(pesq_base)
                    pesq_scores.append(pesq_enh)

                    stoi_score = stoi(clean_sig, enhanced_sig, config.SAMPLE_RATE, extended=False)
                    stoi_scores.append(stoi_score)
                except Exception as e:
                    # print(f"Metric calculation error: {e}")
                    continue

    avg_latency = np.mean(latencies) if latencies else 0
    frame_time_ms = (config.OPUS_FRAME_SIZE / config.SAMPLE_RATE) * 1000 # 10ms

    results = {
        'pesq_baseline': np.mean(pesq_baseline) if pesq_baseline else 0,
        'pesq_enhanced': np.mean(pesq_scores) if pesq_scores else 0,
        'pesq_improvement': np.mean(pesq_scores) - np.mean(pesq_baseline) if pesq_scores else 0,
        'stoi': np.mean(stoi_scores) if stoi_scores else 0,
        'avg_latency_ms': avg_latency, # Latency for one 2-sec chunk
    }
    return results

#%%
# ============================================================================
# CELL 9: PLOTTING FUNCTION
# ============================================================================
def plot_training_history(history):
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # Loss
    axes[0, 0].plot(history['loss'], label='Train Loss')
    axes[0, 0].set_title('Training Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)

    # PESQ
    epochs = history['eval_epochs']
    axes[0, 1].plot(epochs, history['pesq_base'], label='Baseline (Opus)', marker='o')
    axes[0, 1].plot(epochs, history['pesq_enh'], label='Enhanced (Opus+NN)', marker='s')
    axes[0, 1].set_title('PESQ Scores')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('PESQ')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # STOI
    axes[1, 0].plot(epochs, history['stoi'], marker='o', color='blue')
    axes[1, 0].set_title('STOI Score')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('STOI')
    axes[1, 0].grid(True, alpha=0.3)

    # Latency
    axes[1, 1].plot(epochs, history['latency'], marker='o', color='red')
    axes[1, 1].set_title(f'Avg. Latency (per {config.CHUNK_DURATION}s chunk)')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Latency (ms)')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{config.OUTPUT_PATH}/training_history.png", dpi=150)
    plt.show()

#%%
# ============================================================================
# CELL 10: TRAINING LOOP
# ============================================================================
def train_model():
    print("\n" + "="*70)
    print("🎵 ULTRA-CONSERVATIVE OPUS ENHANCEMENT TRAINING")
    print("="*70 + "\n")

    # Initialize
    opus_processor = OPUSProcessor(
        sample_rate=config.SAMPLE_RATE,
        bitrate=config.OPUS_BITRATE,
        frame_size=config.OPUS_FRAME_SIZE
    )

    # Dataset
    print("📂 Loading dataset...")
    full_dataset = AudioEnhancementDataset(
        config.DATASET_PATH,
        opus_processor,
        chunk_duration=config.CHUNK_DURATION,
        sample_rate=config.SAMPLE_RATE,
        max_samples=config.MAX_TRAIN_SAMPLES
    )

    if len(full_dataset) < 10:
        print("Error: Not enough data to train. Stopping.")
        return

    train_size = int(0.9 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, test_size]
    )

    train_loader = DataLoader(
        train_dataset, batch_size=config.BATCH_SIZE, shuffle=True,
        num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=4, shuffle=False,
        num_workers=config.NUM_WORKERS
    )

    # Model
    model = UltraLightEnhancer().to(config.DEVICE)
    print(f"🧠 Ultra-light model: {model.count_parameters():,} parameters")
    print(f"🛡️ Conservative enhancement (alpha max={config.MAX_ALPHA:.2f})")

    # Optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config.NUM_EPOCHS
    )
    criterion = ConservativeLoss(deviation_weight=0.2).to(config.DEVICE)

    # Training
    best_pesq_improvement = -999
    history = {'loss': [], 'pesq_base': [], 'pesq_enh': [], 'stoi': [], 'latency': [], 'eval_epochs': []}
    print("\n🚀 STARTING TRAINING\n")

    for epoch in range(config.NUM_EPOCHS):
        model.train()
        epoch_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS}")

        for degraded, clean in pbar:
            degraded = degraded.to(config.DEVICE)
            clean = clean.to(config.DEVICE)

            enhanced = model(degraded)
            total_loss, clean_loss, deviation = criterion(enhanced, clean, degraded)

            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), config.GRAD_CLIP)
            optimizer.step()

            epoch_loss += total_loss.item()
            pbar.set_postfix({
                'loss': f'{total_loss.item():.4f}',
                'alpha': f'{model.alpha.item():.3f}',
                'lr': f'{scheduler.get_last_lr()[0]:.6f}'
            })

        scheduler.step()
        avg_loss = epoch_loss / len(train_loader)
        history['loss'].append(avg_loss)

        # Evaluate
        if (epoch + 1) % config.EVAL_FREQUENCY == 0:
            print(f"\n📊 Evaluating at epoch {epoch+1}...")
            metrics = evaluate_metrics(model, test_loader, config.DEVICE)
            history['eval_epochs'].append(epoch + 1)
            history['pesq_base'].append(metrics['pesq_baseline'])
            history['pesq_enh'].append(metrics['pesq_enhanced'])
            history['stoi'].append(metrics['stoi'])
            history['latency'].append(metrics['avg_latency_ms'])

            print(f"\n{'='*70}")
            print(f"📈 EPOCH {epoch+1} RESULTS:")
            print(f"  PESQ Baseline:    {metrics['pesq_baseline']:.3f}")
            print(f"  PESQ Enhanced:    {metrics['pesq_enhanced']:.3f}")
            print(f"  PESQ Improvement: {metrics['pesq_improvement']:+.3f} {'✅' if metrics['pesq_improvement'] > 0 else '⚠️'}")
            print(f"  STOI Score:       {metrics['stoi']:.3f}")
            print(f"  Avg. Latency (for {config.CHUNK_DURATION}s chunk): {metrics['avg_latency_ms']:.2f}ms")
            print(f"  Model Alpha:      {model.alpha.item():.3f}")
            print(f"{'='*70}\n")

            # Save if improved
            if metrics['pesq_improvement'] > best_pesq_improvement:
                best_pesq_improvement = metrics['pesq_improvement']
                model_path = f"{config.OUTPUT_PATH}/best_model.pth"
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'metrics': metrics,
                    'epoch': epoch,
                    'config': vars(config) # Save config
                }, model_path)
                print(f"💾 Saved best model to {model_path} (Improvement: {best_pesq_improvement:+.3f})\n")

            # INTERACTIVE CHECK
            if (epoch + 1) == config.INTERACTIVE_CHECKPOINT:
                print("\n" + "="*70)
                print(f"🤔 INTERACTIVE CHECKPOINT - Epoch {epoch+1}")
                print(f"Current PESQ improvement: {metrics['pesq_improvement']:+.3f}")
                try:
                    user_input = input("Continue training? (y/n): ").lower().strip()
                    if user_input == 'n' or user_input == 'no':
                        print("\n⏹️ Stopping training as requested...")
                        break
                    else:
                        print("▶️ Continuing training...\n")
                except EOFError:
                    print("▶️ Non-interactive environment. Continuing training...\n")

    print("✅ Training complete.")
    plot_training_history(history)
    return model, history

#%%
# ============================================================================
# CELL 11: MAIN EXECUTION
# ============================================================================
if __name__ == "__main__":
    model, history = train_model()
    print("\n" + "="*70)
    print("✅ TRAINING COMPLETE!")
    print(f"📁 Best model saved to: {config.OUTPUT_PATH}/best_model.pth")
    print(f"📈 History plot saved to: {config.OUTPUT_PATH}/training_history.png")
    print("="*70 + "\n")

In [12]:
#%%
# ============================================================================
# CELL 1: INSTALLATIONS
# ============================================================================
# Install necessary libraries
!pip install -q pyogg sounddevice pesq pystoi librosa torch torchaudio opuslib soundfile
!apt-get install -qq libopus0 libopus-dev opus-tools

#%%
# ============================================================================
# CELL 2: IMPORTS AND SETUP
# ============================================================================
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
from pesq import pesq
from pystoi import stoi
import opuslib
import soundfile as sf
from google.colab import files
import warnings
import time
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')
print("All libraries loaded.")

#%%
# ============================================================================
# CELL 3: CONFIGURATION (MUST MATCH TRAINING)
# ============================================================================
# This class is needed to re-create the model structure
class Config:
    # Audio parameters
    SAMPLE_RATE = 16000

    # OPUS parameters
    OPUS_BITRATE = 12000
    OPUS_FRAME_SIZE = 160   # 10ms frames
    OPUS_COMPLEXITY = 0

    # Model parameters (MUST MATCH THE SAVED MODEL)
    HIDDEN_SIZE = 32
    KERNEL_SIZE = 3
    MAX_ALPHA = 0.1 # This value is stored in the model's alpha param, but good to have

# Initialize config
config = Config()
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

#%%
# ============================================================================
# CELL 4: OPUS CODEC (MUST MATCH TRAINING)
# ============================================================================
# We need this class to process audio
class OPUSProcessor:
    def __init__(self, sample_rate=16000, bitrate=12000, frame_size=320):
        self.sample_rate = sample_rate
        self.bitrate = bitrate
        self.frame_size = frame_size
        self.encoder = opuslib.Encoder(sample_rate, 1, opuslib.APPLICATION_VOIP)
        self.encoder.bitrate = bitrate
        self.encoder.complexity = config.OPUS_COMPLEXITY
        self.decoder = opuslib.Decoder(sample_rate, 1)

    def process(self, audio):
        try:
            audio = np.clip(audio, -1.0, 1.0)
            audio_int16 = (audio * 32767).astype(np.int16)
            output_frames = []
            for i in range(0, len(audio_int16), self.frame_size):
                frame = audio_int16[i:i + self.frame_size]
                if len(frame) < self.frame_size:
                    frame = np.pad(frame, (0, self.frame_size - len(frame)))
                encoded = self.encoder.encode(frame.tobytes(), self.frame_size)
                decoded = self.decoder.decode(encoded, self.frame_size)
                if decoded is None:
                    decoded_array = np.zeros(self.frame_size, dtype=np.int16)
                else:
                    decoded_array = np.frombuffer(decoded, dtype=np.int16)
                output_frames.append(decoded_array)
            output = np.concatenate(output_frames)[:len(audio_int16)]
            output_float = output.astype(np.float32) / 32767.0
            return np.clip(output_float, -1.0, 1.0)
        except Exception as e:
            print(f"⚠️ OPUS error: {e}")
            return audio

#%%
# ============================================================================
# CELL 5: MODEL DEFINITION (MUST MATCH TRAINING)
# ============================================================================
# We need the exact same class definition to load the saved weights
class UltraLightEnhancer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enhance = nn.Sequential(
            nn.Conv1d(1, config.HIDDEN_SIZE, kernel_size=config.KERNEL_SIZE, padding=config.KERNEL_SIZE//2),
            nn.PReLU(config.HIDDEN_SIZE, init=0.01),
            nn.Conv1d(config.HIDDEN_SIZE, config.HIDDEN_SIZE//2, kernel_size=config.KERNEL_SIZE, padding=config.KERNEL_SIZE//2),
            nn.PReLU(config.HIDDEN_SIZE//2, init=0.01),
            nn.Conv1d(config.HIDDEN_SIZE//2, 1, kernel_size=1),
        )
        # We don't need to init weights, they will be loaded
        self.alpha = nn.Parameter(torch.tensor(0.01)) # Placeholder, will be loaded

    def forward(self, x):
        identity = x
        enhanced_residual = self.enhance(x)
        alpha = torch.clamp(self.alpha, 0.0, config.MAX_ALPHA)
        output = (1 - alpha) * identity + alpha * enhanced_residual
        return torch.clamp(output, -1.0, 1.0)

#%%
# ============================================================================
# CELL 6: HELPER FUNCTIONS
# ============================================================================
def load_model_from_path(model_path, device):
    """Loads the model state_dict from a .pth file."""
    print(f"Loading model from {model_path}...")
    try:
        model = UltraLightEnhancer().to(device)
        # Load the checkpoint
        # FIX: Set weights_only=False due to PyTorch 2.6+ update.
        # Our checkpoint contains a 'metrics' dict, which is safe to load.
        checkpoint = torch.load(model_path, map_location=device, weights_only=False)

        # Extract the state_dict
        if 'model_state_dict' in checkpoint:
            model_state = checkpoint['model_state_dict']
        else:
            model_state = checkpoint

        model.load_state_dict(model_state)
        model.eval()
        print("✅ Model loaded successfully.")

        if 'metrics' in checkpoint:
            print(f"   Loaded model from epoch {checkpoint.get('epoch', 'N/A')}")
            print(f"   Original PESQ Imprv: {checkpoint['metrics'].get('pesq_improvement', 0):.3f}")

        return model
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        print("   Make sure the model definition in this script (Cell 5) is")
        print("   IDENTICAL to the one used for training.")
        return None

def load_audio(audio_path, sr=16000):
    """Loads and normalizes audio."""
    try:
        audio, _ = librosa.load(audio_path, sr=sr, mono=True)
        max_val = np.max(np.abs(audio))
        if max_val > 0.01:
            audio = audio / max_val * 0.95
        audio = audio - np.mean(audio)
        print(f"✅ Audio loaded: {audio_path} ({len(audio)/sr:.2f} seconds)")
        return audio
    except Exception as e:
        print(f"❌ Error loading audio {audio_path}: {e}")
        return None

def calculate_metrics(clean_sig, test_sig, sr):
    """Calculates PESQ and STOI."""
    try:
        pesq_score = pesq(sr, clean_sig, test_sig, 'wb')
        stoi_score = stoi(clean_sig, test_sig, sr, extended=False)
        return pesq_score, stoi_score
    except Exception as e:
        # print(f"Metric error: {e}")
        return 0, 0

#%%
# ============================================================================
# CELL 7: UPLOAD FILES
# ============================================================================
print("="*60)
print("STEP 1: Upload your trained model (.pth file)")
print("="*60)
try:
    uploaded_model = files.upload()
    model_path = list(uploaded_model.keys())[0]
except Exception as e:
    print(f"No file uploaded: {e}")
    model_path = None

if model_path:
    print("\n" + "="*60)
    print("STEP 2: Upload your test audio (.wav, .mp3, etc.)")
    print("="*60)
    try:
        uploaded_audio = files.upload()
        audio_path = list(uploaded_audio.keys())[0]
    except Exception as e:
        print(f"No file uploaded: {e}")
        audio_path = None

#%%
# ============================================================================
# CELL 8: INITIALIZE MODEL AND PROCESSORS
# ============================================================================
if model_path and audio_path:
    # Load the model
    model = load_model_from_path(model_path, DEVICE)

    # Load the audio
    clean_audio = load_audio(audio_path, sr=config.SAMPLE_RATE)

    # Initialize Opus
    opus_processor = OPUSProcessor(
        sample_rate=config.SAMPLE_RATE,
        bitrate=config.OPUS_BITRATE,
        frame_size=config.OPUS_FRAME_SIZE
    )

    if model and clean_audio is not None:
        # Create baseline degraded (Opus only)
        print("\nProcessing with Opus (Baseline)...")
        degraded_audio = opus_processor.process(clean_audio)

        # Save output files
        sf.write('output_01_clean.wav', clean_audio, config.SAMPLE_RATE)
        sf.write('output_02_degraded_opus.wav', degraded_audio, config.SAMPLE_RATE)
        print("Saved 'output_01_clean.wav' and 'output_02_degraded_opus.wav'")

    else:
        print("Skipping tests due to loading errors.")

#%%
# ============================================================================
# CELL 9: TEST 1: FULL AUDIO TEST
# ============================================================================
# This test processes the entire file at once.
# It's good for checking quality but NOT for latency.

if model and clean_audio is not None:
    print("\n" + "="*70)
    print("🔬 TEST 1: FULL AUDIO PROCESSING (Quality Check)")
    print("="*70)

    with torch.no_grad():
        # Prepare tensor
        degraded_tensor = torch.FloatTensor(degraded_audio).unsqueeze(0).unsqueeze(0).to(DEVICE)

        print("Running Opus+NN enhancement (full file)...")
        if DEVICE == 'cuda': torch.cuda.synchronize()
        start_time = time.perf_counter()

        # Run model
        enhanced_tensor = model(degraded_tensor)

        if DEVICE == 'cuda': torch.cuda.synchronize()
        proc_time = (time.perf_counter() - start_time) * 1000 # ms

        # Get result
        enhanced_audio_full = enhanced_tensor.squeeze().cpu().numpy()

        # Save output
        sf.write('output_03_enhanced_full.wav', enhanced_audio_full, config.SAMPLE_RATE)
        print("Saved 'output_03_enhanced_full.wav'")

        # Calculate Metrics
        p_base, s_base = calculate_metrics(clean_audio, degraded_audio, config.SAMPLE_RATE)
        p_enh, s_enh = calculate_metrics(clean_audio, enhanced_audio_full, config.SAMPLE_RATE)

        print("\n--- Full Audio Results ---")
        print(f"Processing time: {proc_time:.2f} ms")
        print(f"| Metric | Baseline (Opus) | Enhanced (Opus+NN) | Improvement |")
        print(f"|--------|-----------------|----------------------|-------------|")
        print(f"| PESQ   | {p_base:15.3f} | {p_enh:20.3f} | {p_enh - p_base:+11.3f} |")
        print(f"| STOI   | {s_base:15.3f} | {s_enh:20.3f} | {s_enh - s_base:+11.3f} |")

#%%
# ============================================================================
# CELL 10: TEST 2: CHUNKED AUDIO TEST (REAL-TIME SIMULATION)
# ============================================================================
# This test simulates a real-time call.
# It processes the audio in 20ms chunks (320 samples) one by one.
# This is the correct way to measure end-to-end latency.

if model and clean_audio is not None:
    print("\n" + "="*70)
    print("🔬 TEST 2: CHUNKED AUDIO PROCESSING (Real-Time Latency Check)")
    print("="*70)

    chunk_size = config.OPUS_FRAME_SIZE # 320 samples (20ms)
    total_model_time_ms = 0

    # Create an empty array to store the output
    enhanced_audio_chunked = np.zeros_like(degraded_audio)

    print(f"Processing audio in {chunk_size}-sample (10ms) chunks...")

    with torch.no_grad():
        # Loop over the degraded audio in chunks
        for i in tqdm(range(0, len(degraded_audio), chunk_size)):
            # Get chunk
            chunk_in = degraded_audio[i:i + chunk_size]

            # Pad if it's the last chunk
            if len(chunk_in) < chunk_size:
                chunk_in = np.pad(chunk_in, (0, chunk_size - len(chunk_in)))

            # Convert to tensor
            chunk_tensor = torch.FloatTensor(chunk_in).unsqueeze(0).unsqueeze(0).to(DEVICE)

            # --- Measure Latency ---
            if DEVICE == 'cuda': torch.cuda.synchronize()
            start_time = time.perf_counter()

            chunk_out = model(chunk_tensor)

            if DEVICE == 'cuda': torch.cuda.synchronize()
            total_model_time_ms += (time.perf_counter() - start_time) * 1000
            # --- End Measure ---

            # Convert back to numpy
            chunk_out_np = chunk_out.squeeze().cpu().numpy()

            # Put in output array (and un-pad)
            len_to_copy = min(len(chunk_out_np), len(enhanced_audio_chunked) - i)
            enhanced_audio_chunked[i:i + len_to_copy] = chunk_out_np[:len_to_copy]

    # Save final chunked output
    sf.write('output_04_enhanced_chunked.wav', enhanced_audio_chunked, config.SAMPLE_RATE)
    print("Saved 'output_04_enhanced_chunked.wav'")

    # --- Calculate Latency Metrics ---
    total_audio_duration_ms = (len(clean_audio) / config.SAMPLE_RATE) * 1000
    num_chunks = len(range(0, len(degraded_audio), chunk_size))
    avg_latency_per_chunk_ms = total_model_time_ms / num_chunks

    # Real-Time Factor (RTF): Audio Duration / Processing Time
    # If RTF > 1.0, it's faster than real-time.
    rtf = total_audio_duration_ms / total_model_time_ms

    print("\n--- Chunked Latency Report ---")
    print(f"Total Audio Duration:   {total_audio_duration_ms:.2f} ms")
    print(f"Total Model Proc. Time: {total_model_time_ms:.2f} ms")
    print(f"Number of Chunks:       {num_chunks}")
    print(f"Average Latency/Chunk:  {avg_latency_per_chunk_ms:.3f} ms (Target for 10ms frame: < 10ms)")
    print(f"Real-Time Factor (RTF): {rtf:.2f}x (Higher is better, >1.0 is real-time)")

    if rtf > 1.0:
        print("✅ SUCCESS: Model processing is faster than real-time.")
    else:
        print("❌ WARNING: Model processing is slower than real-time.")

    # --- Calculate Quality Metrics (for the chunked version) ---
    p_base, s_base = calculate_metrics(clean_audio, degraded_audio, config.SAMPLE_RATE)
    p_enh, s_enh = calculate_metrics(clean_audio, enhanced_audio_chunked, config.SAMPLE_RATE)

    print("\n--- Chunked Quality Results ---")
    print(f"| Metric | Baseline (Opus) | Enhanced (Chunked) | Improvement |")
    print(f"|--------|-----------------|--------------------|-------------|")
    print(f"| PESQ   | {p_base:15.3f} | {p_enh:18.3f} | {p_enh - p_base:+11.3f} |")
    print(f"| STOI   | {s_base:15.3f} | {s_enh:18.3f} | {s_enh - s_base:+11.3f} |")
    print("\n" + "="*70)
    print("✅ All tests complete. You can now listen to the 'output_...wav' files.")
    print("="*70)

elif not model_path:
    print("\nTest script not run. Please upload a model in Cell 7.")
else:
    print("\nTest script not run. Please upload audio in Cell 7.")

All libraries loaded.
Using device: cuda
STEP 1: Upload your trained model (.pth file)


Saving best_model (5).pth to best_model (5).pth

STEP 2: Upload your test audio (.wav, .mp3, etc.)


Saving LJ001-0001.wav to LJ001-0001 (2).wav
Loading model from best_model (5).pth...
✅ Model loaded successfully.
   Loaded model from epoch 24
   Original PESQ Imprv: 0.008
✅ Audio loaded: LJ001-0001 (2).wav (9.66 seconds)

Processing with Opus (Baseline)...
Saved 'output_01_clean.wav' and 'output_02_degraded_opus.wav'

🔬 TEST 1: FULL AUDIO PROCESSING (Quality Check)
Running Opus+NN enhancement (full file)...
Saved 'output_03_enhanced_full.wav'

--- Full Audio Results ---
Processing time: 1.51 ms
| Metric | Baseline (Opus) | Enhanced (Opus+NN) | Improvement |
|--------|-----------------|----------------------|-------------|
| PESQ   |           2.257 |                2.262 |      +0.005 |
| STOI   |           0.889 |                0.889 |      +0.000 |

🔬 TEST 2: CHUNKED AUDIO PROCESSING (Real-Time Latency Check)
Processing audio in 160-sample (10ms) chunks...


  0%|          | 0/966 [00:00<?, ?it/s]

Saved 'output_04_enhanced_chunked.wav'

--- Chunked Latency Report ---
Total Audio Duration:   9655.06 ms
Total Model Proc. Time: 418.26 ms
Number of Chunks:       966
Average Latency/Chunk:  0.433 ms (Target for 10ms frame: < 10ms)
Real-Time Factor (RTF): 23.08x (Higher is better, >1.0 is real-time)
✅ SUCCESS: Model processing is faster than real-time.

--- Chunked Quality Results ---
| Metric | Baseline (Opus) | Enhanced (Chunked) | Improvement |
|--------|-----------------|--------------------|-------------|
| PESQ   |           2.257 |              2.276 |      +0.019 |
| STOI   |           0.889 |              0.889 |      -0.000 |

✅ All tests complete. You can now listen to the 'output_...wav' files.
